##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma 2: Smart Contract Auditing & Explanation

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Smart_Contract_Auditing.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>
<br clear="left"/>

Web3 development requires a high degree of correctness because smart contracts are immutable and often handle significant financial assets. Even small bugs can lead to massive losses.

This notebook demonstrates how to use **Gemma 2 9B IT** as an AI-powered coding assistant for Ethereum development. To fit the 9B model on a standard Colab T4 GPU, we utilize 4-bit quantization using the Hugging Face `transformers` and `bitsandbytes` libraries.

You will learn how to:
1.  Explain complex Solidity code in plain English.
2.  Audit a smart contract to identify specific security vulnerabilities (e.g., Reentrancy).
3.  Refactor the code to apply security best practices.

**Author**: Rahul Lashkari  
**GitHub**: [github.com/rahul-lashkari](https://github.com/rahul-lashkari)  

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources. We will use a **T4 GPU**.

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Hugging Face Setup

To use the 9B model on a T4 GPU efficiently, we will use the Hugging Face hub.

1. **Create a Hugging Face Account:** If you don't already have one, sign up at [huggingface.co](https://huggingface.co/join).
2. **Accept Model Terms:** Visit the [Gemma 2 model page](https://huggingface.co/google/gemma-2-9b-it) and accept the usage conditions.
3. **Get Token:** Go to your [Settings > Access Tokens](https://huggingface.co/settings/tokens) and create a new token.

### Configure your credentials

Add your Hugging Face token to the Colab Secrets manager.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel.
2. Create a new secret: `HF_TOKEN`
3. Copy/paste your token into the Value box.
4. Toggle the button on the left to allow notebook access.

In [2]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies
We install `transformers`, `bitsandbytes` (for quantization), and `accelerate`.

In [3]:
!pip install -q -U torch transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 552.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

## Load Gemma 2 (9B) with Quantization

We load the `google/gemma-2-9b-it` model. To fit it into the 16GB VRAM of a T4 GPU, we use **4-bit NF4 quantization**.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2-9b-it"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    token=os.environ["HF_TOKEN"]
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

### Define Helper Function

We create a helper function to format prompts using the chat template and generate responses.

In [5]:
from IPython.display import Markdown, display

def generate_response(prompt, max_new_tokens=1024):
    messages = [
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens)

    # Decode only the generated response (skipping the prompt)
    response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return response

## The Vulnerable Contract

Let's define a sample Solidity contract. This contract implements a simple "Bank" where users can deposit and withdraw Ether. However, it contains a critical security flaw known as a **Reentrancy Vulnerability**.

In [6]:
vulnerable_contract = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract VulnerableBank {
    mapping(address => uint256) public balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw() public {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "Insufficient balance");

        // Vulnerable line: External call before state update
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");

        balances[msg.sender] = 0;
    }

    function getBalance() public view returns (uint256) {
        return balances[msg.sender];
    }
}
"""

print("Contract loaded into memory.")

Contract loaded into memory.


## Task 1: Code Explanation

First, let's ask Gemma to explain what this contract does for a developer who might be new to Solidity.

In [7]:
prompt_explain = f"""
You are a senior blockchain developer.
Explain the functionality of the following Solidity contract to a junior developer.
Focus on the logic flow of the deposit and withdraw functions.

Code:
{vulnerable_contract}
"""

response = generate_response(prompt_explain)
display(Markdown(response))

Hey there! Let's break down this Solidity contract, focusing on the `deposit` and `withdraw` functions.

**Deposit Function (`deposit`)**

1. **`public payable`**: This means anyone can call this function, and it can accept Ether (ETH) as input.

2. **`balances[msg.sender] += msg.value;`**: This is the core logic. 
   - `msg.sender` represents the address of the person calling the function (the depositor).
   - `msg.value` holds the amount of ETH sent with the function call.
   - We're adding `msg.value` to the depositor's balance stored in the `balances` mapping.

**Withdraw Function (`withdraw`)**

1. **`uint256 balance = balances[msg.sender];`**: We first retrieve the depositor's current balance from the `balances` mapping.

2. **`require(balance > 0, "Insufficient balance");`**: This is a safety check. If the balance is zero or less, the function will revert with the error message "Insufficient balance".

3. **` (bool success, ) = msg.sender.call{value: balance}("");`**: This is where things get tricky. This line is **vulnerable**.
   - It's an external call to the depositor's address (`msg.sender`).
   - `call{value: balance}("")` sends the depositor's entire balance (`balance`) to their address.
   - The `success` variable tells us if the call was successful.

4. **`require(success, "Transfer failed");`**: If the external call fails (e.g., the depositor's address is invalid), the function will revert with the error message "Transfer failed".

5. **`balances[msg.sender] = 0;`**: Finally, we set the depositor's balance to zero, indicating that the withdrawal was successful.

**The Vulnerability**

The problem lies in the order of operations. The external call (`msg.sender.call`) happens **before** the balance is updated (`balances[msg.sender] = 0`). This creates a window of opportunity for an attacker to exploit the contract.

**Exploitation Scenario**

1. An attacker calls the `withdraw` function.
2. The contract retrieves the attacker's balance but hasn't yet updated it to zero.
3. The contract sends the attacker's balance to their address.
4. The attacker can then call the `withdraw` function again, potentially withdrawing the same funds twice.

**Mitigation**

To fix this vulnerability, the balance update should happen **before** the external call. This ensures that the balance is accurately reflected before any funds are transferred.



Let me know if you have any more questions!


## Task 2: Security Audit

Now, let's perform a security audit. We specifically ask the model to look for common Web3 attack vectors.

In [8]:
prompt_audit = f"""
Act as a Smart Contract Security Auditor.
Analyze the provided Solidity code for security vulnerabilities.
Identify the specific type of attack this contract is susceptible to and explain how an attacker could exploit it.

Code:
{vulnerable_contract}
"""

response = generate_response(prompt_audit)
display(Markdown(response))

This contract is susceptible to a **reentrancy attack**. 

Here's how an attacker could exploit it:

1. **Create a malicious contract:** The attacker would create a new contract that implements a fallback function. This fallback function would be designed to call the `withdraw` function of the `VulnerableBank` contract again, recursively.

2. **Deposit funds:** The attacker would deposit some funds into the `VulnerableBank` contract.

3. **Call withdraw:** The attacker would then call the `withdraw` function on the `VulnerableBank` contract.

4. **Exploit the vulnerability:**  The crucial vulnerability lies in the line `(bool success, ) = msg.sender.call{value: balance}("");`.  Here's why:

   * **External call before state update:** The `call` function to send the funds to the attacker's address is executed *before* the `balances[msg.sender] = 0;` line. This means that the attacker's contract can still call the `withdraw` function again, even though the `VulnerableBank` contract thinks the withdrawal has already been completed.

   * **Recursive calls:** The attacker's fallback function will call `withdraw` again, draining the attacker's balance in the `VulnerableBank` contract. This process can repeat multiple times, allowing the attacker to steal all the funds.

5. **Profit:** The attacker's contract will receive the funds, leaving the `VulnerableBank` contract empty.

**Mitigation:**

To prevent reentrancy attacks, you should always update the contract's state *before* making any external calls. One common technique is to use a reentrancy guard, which is a boolean variable that is set to `true` before an external call and `false` afterwards. This prevents the attacker from calling the same function again during the same transaction.



Let me know if you have any other Solidity code you'd like me to analyze!


## Task 3: Secure Refactoring

Finally, let's ask Gemma to rewrite the contract to be secure, applying the "Checks-Effects-Interactions" pattern.

In [9]:
prompt_fix = f"""
Rewrite the 'VulnerableBank' contract to fix the reentrancy vulnerability.
Apply the Checks-Effects-Interactions pattern.
Add comments explaining the changes.

Original Code:
{vulnerable_contract}
"""

response = generate_response(prompt_fix)
display(Markdown(response))

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract SecureBank {
    mapping(address => uint256) public balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw() public {
        // Checks
        uint256 balance = balances[msg.sender];
        require(balance > 0, "Insufficient balance");

        // Effects
        balances[msg.sender] = 0; // Update balance first

        // Interactions
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");
    }

    function getBalance() public view returns (uint256) {
        return balances[msg.sender];
    }
}
```

**Explanation of Changes:**

1. **Checks-Effects-Interactions Pattern:** The `withdraw` function now strictly adheres to the Checks-Effects-Interactions pattern:
   - **Checks:** The function first checks if the user has sufficient funds (`balance > 0`).
   - **Effects:** The balance of the user is updated to 0 *before* any external calls are made. This ensures that the reentrancy vulnerability is prevented.
   - **Interactions:** The external call to the user's address (`msg.sender.call{value: balance}("")`) is made only after the balance has been updated.

2. **Reentrancy Prevention:** By updating the balance *before* making the external call, we prevent a malicious contract from calling back into the `withdraw` function after the balance has been reduced but before the state change is finalized.

**How Reentrancy Works:**

- A malicious contract could call the `withdraw` function.
- The `withdraw` function would then call the malicious contract's `fallback` function, which could contain code to re-enter the `withdraw` function again.
- This recursive call would allow the attacker to drain the bank's funds repeatedly, even if their balance is initially insufficient.

**The `Checks-Effects-Interactions` pattern ensures that the state of the contract is updated before any external calls are made, preventing this type of attack.**


## Conclusion

In this notebook, you used **Gemma 2 (9B)** to act as a full-stack Web3 assistant. You successfully:
1.  Analyzed a raw Solidity file.
2.  Identified a critical Reentrancy bug.
3.  Generated a secure patch for the code using security best practices.